In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline
import matplotlib
import time

In [ ]:
d=4 

L = (2.4*d+4)*1000
f = 470 #THz
dnu = 3e-3 #THz
dL = dnu/f*L #in nm
print dL

In [ ]:
conversion = 65
dV = dL/conversion
print dV

In [ ]:
print 20./500

In [ ]:
Vpp = 20.
dL = Vpp*conversion
dnu = dL/L*f
print dnu, 'GHZ'

In [ ]:
folder= tb.latest_data('Giuseppe')#r'D:\measuring\data\20171207\135353_pidcavl'
#r'D:\measuring\data\20171207\133039_pidcavl'#r'D:\measuring\data\20171207\130217_pidcavl'#r'D:\measuring\data\20171206\171035_pidcavl'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=14)

V,ZPL,x,PSB = d[:,0],d[:,1],d[:,2],d[:,3]


In [ ]:
import math
integrationtime=1#seconds

fig,ax=plt.subplots()
ax.plot(V,ZPL/integrationtime,'o')
ax.set_ylabel('ZPL counts (Hz)',color='b',fontsize=14)
ax.set_xlabel('JPE voltage (V)')
g_a = 0
g_A=1200
g_x0=1.97#2.75#1.55
g_gamma=0.01
g_A2=100
g_x02=1.98#2.65#1.45
g_gamma2=0.04
fixed=[]
ax.set_title(folder)


print 'max:',max(ZPL)/integrationtime, ';   min:',min(ZPL)/integrationtime, ';   ~ S/N:', max(ZPL)/min(ZPL)
ax3=ax.twinx()
ax3.plot(V,PSB,'g')
ax3.set_ylabel('PSB',color='g')
# ax3.set_xlim(1.95,2.0)

plt.show()

ZPL_zoom = ZPL#[120:160]
V_zoom  = V#[120:160]

p0,fitfunc,fitfunc_str=common.fit_2lorentz(g_a,g_A,g_x0,g_gamma,g_A2,g_x02,g_gamma2)
fit_res=fit.fit1d(V,ZPL/integrationtime,fitmethod=None,p0=p0,fitfunc=fitfunc,fitfunc_str=fitfunc_str,fixed=fixed)

p0,fitfunc,fitfunc_str=common.fit_2lorentz(g_a,g_A,g_x0,g_gamma,g_A2,g_x02,g_gamma2)
fit_res_zoom=fit.fit1d(V_zoom,ZPL_zoom/integrationtime,fitmethod=None,p0=p0,fitfunc=fitfunc,fitfunc_str=fitfunc_str,fixed=fixed)

def plot_fitted_lorentzians(fit_res, V):
    ax2 = plot.plot_fit1d(fit_res,ret='ax',print_info=False)
    ax2.set_ylabel('ZPL counts (Hz)',color='b',fontsize=14)
    ax2.set_xlabel('JPE voltage (V)',fontsize=14)
    gamma=  fit_res['params_dict']['gamma1']
    x0=  fit_res['params_dict']['x01']
    A=  fit_res['params_dict']['A1']
    gamma2=  fit_res['params_dict']['gamma2']
    x02=  fit_res['params_dict']['x02']
    A2=  fit_res['params_dict']['A2']
    a=  fit_res['params_dict']['a1']
    print gamma,x0,A,a

    max1 = 2.*A/(np.pi*gamma)+a
    max2 = 2.*A2/(np.pi*gamma2)+a
    ax2.plot([x0-gamma/2,x0+gamma/2],[1.*A/(np.pi*gamma)+a,1.*A/(math.pi*gamma)+a],'g',linewidth=3)
    ax2.plot([x02-gamma2/2,x02+gamma2/2],[1.*A2/(np.pi*gamma2)+a,1.*A2/(math.pi*gamma2)+a],color='c',linewidth=3)
    ax2.plot(V,a+2*A/math.pi*gamma/(4*(V-x0)**2+gamma**2),'g')
    ax2.plot(V,a+2*A2/math.pi*gamma2/(4*(V-x02)**2+gamma2**2),'c')
    ax2.set_title(folder)
    plt.show()
plot_fitted_lorentzians(fit_res, V)


print 'linewidth = ',round(gamma,3),'V'
print 'linewidth2 = ',round(gamma2,3),'V'